In [2]:
#importing the necessary Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings


In [3]:
#reading the loan.csv data
data=pd.read_csv("loan.csv",index_col=0)
count=0
total_missing_value_cols=[]
for col in data.columns:
    if data[col].isnull().sum() == 39717:
        print(col,data[col].isnull().count())
        count+=1
        total_missing_value_cols.append(col)

print(count)

/tmp/ipykernel_171625/2769543212.py:1: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("loan.csv",index_col=0)


mths_since_last_major_derog 39717
annual_inc_joint 39717
dti_joint 39717
verification_status_joint 39717
tot_coll_amt 39717
tot_cur_bal 39717
open_acc_6m 39717
open_il_6m 39717
open_il_12m 39717
open_il_24m 39717
mths_since_rcnt_il 39717
total_bal_il 39717
il_util 39717
open_rv_12m 39717
open_rv_24m 39717
max_bal_bc 39717
all_util 39717
total_rev_hi_lim 39717
inq_fi 39717
total_cu_tl 39717
inq_last_12m 39717
acc_open_past_24mths 39717
avg_cur_bal 39717
bc_open_to_buy 39717
bc_util 39717
mo_sin_old_il_acct 39717
mo_sin_old_rev_tl_op 39717
mo_sin_rcnt_rev_tl_op 39717
mo_sin_rcnt_tl 39717
mort_acc 39717
mths_since_recent_bc 39717
mths_since_recent_bc_dlq 39717
mths_since_recent_inq 39717
mths_since_recent_revol_delinq 39717
num_accts_ever_120_pd 39717
num_actv_bc_tl 39717
num_actv_rev_tl 39717
num_bc_sats 39717
num_bc_tl 39717
num_il_tl 39717
num_op_rev_tl 39717
num_rev_accts 39717
num_rev_tl_bal_gt_0 39717
num_sats 39717
num_tl_120dpd_2m 39717
num_tl_30dpd 39717
num_tl_90g_dpd_24m 39717


In [4]:
data.shape
data_reformed=data.copy()
data_reformed=data_reformed.drop(total_missing_value_cols,axis=1)

In [5]:
data_reformed.shape
data_reformed.isnull().sum().sort_values(ascending=False)
cols_with_above_30_null_values=[]
for col in data_reformed.columns:
    missing_val_perc=100*(data_reformed[col].isnull().sum()/data_reformed.shape[0])
    if missing_val_perc>30:
        print("{} column with {}% missing values".format(col,missing_val_perc))
        cols_with_above_30_null_values.append(col)

desc column with 32.58050708764509% missing values
mths_since_last_delinq column with 64.66248709620565% missing values
mths_since_last_record column with 92.98537150338646% missing values
next_pymnt_d column with 97.12969257496789% missing values


In [6]:

data_reformed=data_reformed.drop(cols_with_above_30_null_values,axis=1)

In [7]:
data_reformed.shape

(39717, 52)

In [11]:
missing_perc=round(100*(data_reformed.isnull().sum()/data_reformed.shape[0]),3)
missing_perc[missing_perc>0]

emp_title                     6.191
emp_length                    2.707
title                         0.028
revol_util                    0.126
last_pymnt_d                  0.179
last_credit_pull_d            0.005
collections_12_mths_ex_med    0.141
chargeoff_within_12_mths      0.141
pub_rec_bankruptcies          1.755
tax_liens                     0.098
dtype: float64

In [17]:
def print_unique_valuecount(df,col):
    print(f'{col} with {len(df[col].unique())} number of unique values')
cols=["emp_title","emp_length","title","revol_util","last_pymnt_d",
      "last_credit_pull_d","collections_12_mths_ex_med",
      "chargeoff_within_12_mths","pub_rec_bankruptcies","tax_liens"]
for col in cols:
    print_unique_valuecount(data_reformed,col)


emp_title with 28821 number of unique values
emp_length with 12 number of unique values
title with 19616 number of unique values
revol_util with 1090 number of unique values
last_pymnt_d with 102 number of unique values
last_credit_pull_d with 107 number of unique values
collections_12_mths_ex_med with 2 number of unique values
chargeoff_within_12_mths with 2 number of unique values
pub_rec_bankruptcies with 4 number of unique values
tax_liens with 2 number of unique values


AttributeError: 'DataFrame' object has no attribute 'unique'

In [77]:
data_reformed.nunique().sort_values(ascending=False)

member_id                     39717
url                           39717
total_pymnt                   37850
total_pymnt_inv               37518
total_rec_int                 35148
last_pymnt_amnt               34930
emp_title                     28820
revol_bal                     21711
title                         19615
installment                   15383
funded_amnt_inv                8205
total_rec_prncp                7976
annual_inc                     5318
recoveries                     4040
dti                            2868
collection_recovery_fee        2616
total_rec_late_fee             1356
out_prncp_inv                  1138
out_prncp                      1137
revol_util                     1089
funded_amnt                    1041
loan_amnt                       885
zip_code                        823
earliest_cr_line                526
int_rate                        371
last_credit_pull_d              106
last_pymnt_d                    101
total_acc                   

We are observing that the columnns namely
acc_now_delinq,
delinq_amnt,
chargeoff_within_12_mths,
initial_list_status,
application_type,
policy_code,
collections_12_mths_ex_med,
pymnt_plan,
tax_liens

are categorical variables with only one unique value which doesn't provide much to the analysis of the loan status both in the case of univariate and bivariate analysis
so we can remove these columns

Also member_id and url variable are total unique values and we can remove them as well


In [21]:
cols_to_be_removed=["acc_now_delinq", "delinq_amnt", "chargeoff_within_12_mths",
                    "initial_list_status", "application_type", "policy_code"
                    , "collections_12_mths_ex_med", "pymnt_plan", "tax_liens","member_id","url"]
data_reformed=data_reformed.drop(cols_to_be_removed,axis=1)
data_reformed.shape

(39717, 41)

In [23]:
data_reformed.isna().sum()

loan_amnt                     0
funded_amnt                   0
funded_amnt_inv               0
term                          0
int_rate                      0
installment                   0
grade                         0
sub_grade                     0
emp_title                  2459
emp_length                 1075
home_ownership                0
annual_inc                    0
verification_status           0
issue_d                       0
loan_status                   0
purpose                       0
title                        11
zip_code                      0
addr_state                    0
dti                           0
delinq_2yrs                   0
earliest_cr_line              0
inq_last_6mths                0
open_acc                      0
pub_rec                       0
revol_bal                     0
revol_util                   50
total_acc                     0
out_prncp                     0
out_prncp_inv                 0
total_pymnt                   0
total_py

In [24]:
data_reformed.to_csv("semi_cleaned_loan.csv",encoding="utf-8",index=False)